In [1]:
#!pip install tensorflow-gpu==2.4.0 #2,3,4,7,8,11
#!pip install numpy==1.19.5

In [2]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from google.colab import drive 
from scipy.optimize import brentq
np.random.seed(2022)
import random

random.seed(2022)
from scipy import stats
################################### for neural network modeling
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
tf.random.set_seed(2022)
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
######################################################
### stationary poisson process
######################################################
def generate_stationary_poisson():
    tau = np.random.exponential(size=100000)
    T = tau.cumsum()
    score = 1
    return [T,score]

######################################################
### non-stationary poisson process
######################################################
def generate_nonstationary_poisson():
    L = 20000
    amp = 0.99
    l_t = lambda t: np.sin(2*np.pi*t/L)*amp + 1
    l_int = lambda t1,t2: - L/(2*np.pi)*( np.cos(2*np.pi*t2/L) - np.cos(2*np.pi*t1/L) )*amp   + (t2-t1)
    
    while 1:
        T = np.random.exponential(size=210000).cumsum()*0.5
        r = np.random.rand(210000)
        index = r < l_t(T)/2.0
        
        if index.sum() > 100000:
            T = T[index][:100000]
            score = - ( np.log(l_t(T[80000:])).sum() - l_int(T[80000-1],T[-1]) )/20000
            break
       
    return [T,score]

######################################################
### stationary renewal process
######################################################
def generate_stationary_renewal():
    s = np.sqrt(np.log(6*6+1))
    mu = -s*s/2
    tau = lognorm.rvs(s=s,scale=np.exp(mu),size=100000)
    lpdf = lognorm.logpdf(tau,s=s,scale=np.exp(mu))
    T = tau.cumsum()
    score = - np.mean(lpdf[80000:])
    
    return [T,score]
   
######################################################
### non-stationary renewal process
######################################################
def generate_nonstationary_renewal():
    L = 20000
    amp = 0.99
    l_t = lambda t: np.sin(2*np.pi*t/L)*amp + 1
    l_int = lambda t1,t2: - L/(2*np.pi)*( np.cos(2*np.pi*t2/L) - np.cos(2*np.pi*t1/L) )*amp   + (t2-t1)

    T = []
    lpdf = []
    x = 0

    k = 4
    rs = gamma.rvs(k,size=100000)
    lpdfs = gamma.logpdf(rs,k)
    rs = rs/k
    lpdfs = lpdfs + np.log(k)

    for i in range(100000):
        x_next = brentq(lambda t: l_int(x,t) - rs[i],x,x+1000)
        l = l_t(x_next)
        T.append(x_next)
        lpdf.append(  lpdfs[i] + np.log(l) )  
        x = x_next

    T = np.array(T)
    lpdf = np.array(lpdf)
    score = - lpdf[80000:].mean()
    
    return [T,score]
 
######################################################
### self-correcting process
######################################################
def generate_self_correcting():
    
    def self_correcting_process(mu,alpha,n):
    
        t = 0; x = 0;
        T = [];
        log_l = [];
        Int_l = [];
    
        for i in range(n):
            e = np.random.exponential()
            tau = np.log( e*mu/np.exp(x) + 1 )/mu # e = ( np.exp(mu*tau)- 1 )*np.exp(x) /mu
            t = t+tau
            T.append(t)
            x = x + mu*tau
            log_l.append(x)
            Int_l.append(e)
            x = x -alpha

        return [np.array(T),np.array(log_l),np.array(Int_l)]
    
    [T,log_l,Int_l] = self_correcting_process(1,1,100000)
    score = - ( log_l[80000:] - Int_l[80000:] ).sum() / 20000
    
    return [T,score]

######################################################
### Hawkes process
######################################################
def generate_hawkes1():
    [T,LL] = simulate_hawkes(100000,0.2,[0.8,0.0],[1.0,20.0])
    score = - LL[80000:].mean()
    return [T,score]

def generate_hawkes2():
    [T,LL] = simulate_hawkes(100000,0.2,[0.4,0.4],[1.0,20.0])
    score = - LL[80000:].mean()
    return [T,score]

def simulate_hawkes(n,mu,alpha,beta):
    T = []
    LL = []
    
    x = 0
    l_trg1 = 0
    l_trg2 = 0
    l_trg_Int1 = 0
    l_trg_Int2 = 0
    mu_Int = 0
    count = 0
    
    while 1:
        l = mu + l_trg1 + l_trg2
        step = np.random.exponential()/l
        x = x + step
        
        l_trg_Int1 += l_trg1 * ( 1 - np.exp(-beta[0]*step) ) / beta[0]
        l_trg_Int2 += l_trg2 * ( 1 - np.exp(-beta[1]*step) ) / beta[1]
        mu_Int += mu * step
        l_trg1 *= np.exp(-beta[0]*step)
        l_trg2 *= np.exp(-beta[1]*step)
        l_next = mu + l_trg1 + l_trg2
        
        if np.random.rand() < l_next/l: #accept
            T.append(x)
            LL.append( np.log(l_next) - l_trg_Int1 - l_trg_Int2 - mu_Int )
            l_trg1 += alpha[0]*beta[0]
            l_trg2 += alpha[1]*beta[1]
            l_trg_Int1 = 0
            l_trg_Int2 = 0
            mu_Int = 0
            count += 1
            
            if count == n:
                break
        
    return [np.array(T),np.array(LL)]


In [4]:
def _compute_gradients(tensor, var_list):
  grads = tf.gradients(tensor, var_list)
  return [grad if grad is not None else tf.zeros_like(var) for var, grad in zip(var_list, grads)]

In [5]:
def generate_fully_ntpp_model(T_train,size_nn,size_rnn):
  ## hyper parameters
  time_step = 20 # truncation depth of RNN 
  #size_rnn = 64 # the number of units in the RNN
  #size_nn = 64 # the nubmer of units in each hidden layer in the cumulative hazard function network
  size_layer_chfn = 2 # the number of the hidden layers in the cumulative hazard function network

  ## mean and std of the log of the inter-event interval, which will be used for the data standardization
  mu = np.log(np.ediff1d(T_train)).mean()
  sigma = np.log(np.ediff1d(T_train)).std()

  ## kernel initializer for positive weights
  def abs_glorot_uniform(shape, dtype=None, partition_info=None): 
      return K.abs(keras.initializers.glorot_uniform(seed=None)(shape,dtype=dtype))

  ## Inputs 
  event_history  = keras.layers.Input(shape=(time_step,1)) # input to RNN (event history)
  elapsed_time = keras.layers.Input(shape=(1,)) # input to cumulative hazard function network (the elapsed time from the most recent event)

  ## log-transformation and standardization
  event_history_nmlz = keras.layers.Lambda(lambda x: (K.log(x)-mu)/sigma )(event_history)
  elapsed_time_nmlz = keras.layers.Lambda(lambda x: (K.log(x)-mu)/sigma )(elapsed_time) 

  ## RNN
  output_rnn = keras.layers.SimpleRNN(size_rnn,activation = 'tanh',return_sequences=False,input_shape=(time_step,1))(event_history_nmlz) # activation was tanh before 

  ## the first hidden layer in the cummulative hazard function network
  hidden_tau = keras.layers.Dense(size_nn,kernel_initializer=abs_glorot_uniform,kernel_constraint=keras.constraints.NonNeg(),use_bias=False)(elapsed_time_nmlz) # elapsed time -> the 1st hidden layer, positive weights
  hidden_rnn = keras.layers.Dense(size_nn)(output_rnn) # rnn output -> the 1st hidden layer
  #hidden = K.concatenate([hidden_tau,hidden_rnn])
  hidden = keras.layers.Lambda(lambda inputs: K.sin(inputs[0]+inputs[1]) )([hidden_tau,hidden_rnn])
  
  ## Outputs
  Int_l = keras.layers.Dense(1,kernel_initializer=abs_glorot_uniform, kernel_constraint=keras.constraints.NonNeg())(hidden) # cumulative hazard function, positive weights
  Int_l = tf.keras.layers.LeakyReLU(alpha = 0.5)(Int_l)
  Int_l = keras.layers.Lambda(lambda x: (x + K.abs(K.min(x))) / K.max(x))(Int_l)

  #l = layers.Lambda( lambda inputs: K.gradients(inputs[0],inputs[1])[0] )([Int_l,elapsed_time]) # hazard function

  l = keras.layers.Lambda( lambda inputs: _compute_gradients(inputs[0],[inputs[1]])[0] )([Int_l,elapsed_time]) # hazard function

  ## define model
  model = Model(inputs=[event_history,elapsed_time],outputs=[l,Int_l])
  model.add_loss( -K.mean( K.log( 1e-10 + l ) - Int_l ) ) # set loss function to be the negative log-likelihood function
  return model


In [6]:
from scipy.stats import (
    norm, beta, expon, gamma, genextreme, logistic, lognorm, triang, uniform, fatiguelife,            
    gengamma, gennorm, dweibull, dgamma, gumbel_r, powernorm, rayleigh, weibull_max, weibull_min, 
    laplace, alpha, genexpon, bradford, betaprime, burr, fisk, genpareto, hypsecant, 
    halfnorm, halflogistic, invgauss, invgamma, levy, loglaplace, loggamma, maxwell, 
    mielke, ncx2, ncf, nct, nakagami, pareto, lomax, powerlognorm, powerlaw, rice, 
    semicircular, rice, invweibull, foldnorm, foldcauchy, cosine, exponpow, 
    exponweib, wald, wrapcauchy, truncexpon, truncnorm, t, rdist
    )

distributions = [
    norm, beta, expon, gamma, genextreme, logistic, lognorm, triang, uniform, fatiguelife,            
    gengamma, gennorm, dweibull, dgamma, gumbel_r, powernorm, rayleigh, weibull_max, weibull_min, 
    laplace, alpha, genexpon, bradford, betaprime, burr, fisk, genpareto, hypsecant, 
    halfnorm, halflogistic, invgauss, invgamma, levy, loglaplace, loggamma, maxwell, 
    mielke, ncx2, ncf, nct, nakagami, pareto, lomax, powerlognorm, powerlaw, rice, 
    semicircular, rice, invweibull, foldnorm, foldcauchy, cosine, exponpow, 
    exponweib, wald, wrapcauchy, truncexpon, truncnorm, t, rdist
    ]

In [7]:
process_type_array = ['stationary_poisson',
                       'nonstationary_poisson',
                       'stationary_renewal',
                       'nonstationary_renewal',
                       'self_correcting',
                       'hawkes1',
                       'hawkes2']

def generate_samples(process_type):
  if process_type == 'stationary_poisson':
    [T,score] = generate_stationary_poisson()
  elif process_type == 'nonstationary_poisson':
    [T,score] = generate_nonstationary_poisson()
  elif process_type == 'stationary_renewal':
    [T,score] = generate_stationary_renewal()
  elif process_type == 'nonstationary_renewal':
    [T,score] = generate_nonstationary_renewal()
  elif process_type == 'self_correcting':
    [T,score] = generate_self_correcting()
  elif process_type == 'hawkes1':
    [T,score] = generate_hawkes1()
  elif process_type == 'hawkes2':
    [T,score] = generate_hawkes2()
  return [T,score]

In [8]:
def compute_p_values(id_samples, ood_samples):
  id_samples = np.array(id_samples)
  p_values = []
  for item in ood_samples:
    p_value = min(len(id_samples[id_samples <= item]), len(id_samples[id_samples >= item]))/len(id_samples)
    p_values.append(p_value)
  return p_values

In [ ]:
########## Hyper-parameters ##############
time_step = 20 # truncation depth of a RNN 
size_rnn = 64  # the number of units in a RNN
size_div = 128 # the number of sub-intervals of the piecewise constant function (piecewise constant model)
size_layer = 2 # the number of hidden layers of the cumulative hazard function network (neural network based model)
id_test_samples = 1000
ood_test_samples = 1000

# for each process we train the model and test it in the same and the other processes 
for i in range(3,len(process_type_array)):
  [T_train, score] = generate_samples(process_type_array[i]) # generate training data 
  fully_ntpp_model = generate_fully_ntpp_model(T_train,size_nn = 32, size_rnn = 128) # model 
  callback = EarlyStopping(monitor='loss', patience=3) # callbacks

  ## format the input data
  dT_train = np.ediff1d(T_train) # transform a series of timestamps to a series of interevent intervals: T_train -> dT_train
  n = dT_train.shape[0] 
  input_RNN = np.array( [ dT_train[z:z+time_step] for z in range(n-time_step) ]).reshape(n-time_step,time_step,1)
  input_CHFN = dT_train[-n+time_step:].reshape(n-time_step,1)

  ## training 
  # learning_rate=3e-5,decay = 5e-5
  fully_ntpp_model.compile(keras.optimizers.Adam(learning_rate=0.001))
  fully_ntpp_model.fit([input_RNN,input_CHFN],epochs=50,batch_size=32,validation_split=0.2, callbacks=[callback],verbose=0)  

  '''we compare against a parameteric approach? maybe gamma is more suited''' 
  params = weibull_min.fit(np.diff(T_train), floc=0) # fit -> estimates the parameters 
  param_names = ["shape", "loc", "shape"]
  #dict_param = {print(k,":",f'{v:.3f}') for k,v in zip(param_names, params)}

  rv = weibull_min(params[0], params[1], params[2])
  weibull_nll = -1*np.mean(np.log(rv.pdf(np.diff(T_train))))
  
  ntpp_id_sample_nll_array = []
  weibull_id_sample_nll_array = []
  for k in range(id_test_samples):
    [T_id, score] = generate_samples(process_type_array[i])

    ## format the input data
    dT_test = np.ediff1d(T_id) # transform a series of timestamps to a series of interevent intervals: T_test -> dT_test
    n = dT_test.shape[0]
    input_RNN_test = np.array([dT_test[z:z+time_step] for z in range(n-time_step)]).reshape(n-time_step,time_step,1)
    input_CHFN_test = dT_test[-n+time_step:].reshape(n-time_step,1)

    ## testing
    [l_test,Int_l_test] = fully_ntpp_model.predict([input_RNN_test,input_CHFN_test],batch_size=input_RNN_test.shape[0])
    LL = np.log(l_test+1e-10) - Int_l_test # log-liklihood
    #print('-LL: ', -LL)
    #print("Mean negative log-likelihood per event: ",-LL.mean())

    ntpp_id_sample_nll_array.append(-LL.mean())

    #rv.pdf(np.diff(T_test))
    weibull_nll = -1*np.mean(np.log(rv.pdf(np.diff(T_id))))
    weibull_id_sample_nll_array.append(weibull_nll)
    
  for j in range(len(process_type_array)):
    ntpp_ood_sample_nll_array = []
    weibull_ood_sample_nll_array = []
    for k in range(ood_test_samples):
      [T_ood, score] = generate_samples(process_type_array[j]) # generate ood data 

      ## format the input data
      dT_test = np.ediff1d(T_ood) # transform a series of timestamps to a series of interevent intervals: T_test -> dT_test
      n = dT_test.shape[0]
      input_RNN_test = np.array([dT_test[z:z+time_step] for z in range(n-time_step)]).reshape(n-time_step,time_step,1)
      input_CHFN_test = dT_test[-n+time_step:].reshape(n-time_step,1)

      ## testing
      [l_test,Int_l_test] = fully_ntpp_model.predict([input_RNN_test,input_CHFN_test],batch_size=input_RNN_test.shape[0])
      LL = np.log(l_test+1e-10) - Int_l_test # log-liklihood
      #print('-LL: ', -LL)
      #print("Mean negative log-likelihood per event: ",-LL.mean())

      ntpp_ood_sample_nll_array.append(-LL.mean())

      weibull_nll = -1*np.mean(np.log(rv.pdf(np.diff(T_ood))))
      weibull_ood_sample_nll_array.append(weibull_nll)

    #estimate p-values
    ntpp_p_values = compute_p_values(ntpp_id_sample_nll_array, ntpp_ood_sample_nll_array)
    weibull_p_values = compute_p_values(weibull_id_sample_nll_array, weibull_ood_sample_nll_array)
    #convert to numpy
    ntpp_p_values = np.array(ntpp_p_values)
    weibull_p_values = np.array(weibull_p_values)
    #print(f"NN_size: {nn_layer} layers, RNN_size: {rnn_layer} ")
    print('id: ', process_type_array[i], ' ood: ', process_type_array[j])
    print("Detection Rate (NTPP): ", len(ntpp_p_values[ntpp_p_values < 0.05]))
    print("Detection Rate (Weibull): ", len(weibull_p_values[weibull_p_values < 0.05]))
    print("-------------------------------------------------------------------------------------------------")

<ipython-input-9-e4a596f278b3>:47: RuntimeWarning: invalid value encountered in log
  LL = np.log(l_test+1e-10) - Int_l_test # log-liklihood
<ipython-input-9-e4a596f278b3>:71: RuntimeWarning: invalid value encountered in log
  LL = np.log(l_test+1e-10) - Int_l_test # log-liklihood


id:  nonstationary_renewal  ood:  stationary_poisson
Detection Rate (NTPP):  1000
Detection Rate (Weibull):  573
-------------------------------------------------------------------------------------------------
